In [1]:
import pandas as pd
import plotly.express as px


In [2]:
f = pd.read_csv("2022_All_Countries_Region_Mobility_Report.csv")
df = pd.DataFrame(f,columns=["country_region_code","date","retail_and_recreation_percent_change_from_baseline","grocery_and_pharmacy_percent_change_from_baseline","parks_percent_change_from_baseline","transit_stations_percent_change_from_baseline","workplaces_percent_change_from_baseline","residential_percent_change_from_baseline"])


In [7]:
# df["retail_and_recreation_percent_change_from_baseline"] = df["retail_and_recreation_percent_change_from_baseline"] + 100
df["retail_and_recreation_percent_change_from_baseline"].drop()

0     100.0
1     101.0
2      99.0
3      98.0
4      98.0
5      98.0
6      97.0
7      98.0
8      99.0
9      97.0
10     97.0
11     98.0
12    101.0
13    101.0
14    102.0
15    103.0
16    100.0
17    100.0
18     99.0
19     97.0
20     93.0
21     97.0
22    101.0
23     97.0
24     96.0
25     96.0
26     92.0
27     84.0
28     88.0
29     94.0
30     89.0
31     84.0
32     81.0
33     76.0
34     63.0
35     68.0
36     70.0
37     73.0
38     61.0
39     50.0
40     45.0
41     20.0
42     28.0
43     45.0
44     43.0
45     44.0
46     44.0
47     42.0
48     31.0
49     42.0
Name: retail_and_recreation_percent_change_from_baseline, dtype: float64

In [6]:
fig = px.scatter_geo(df, locations="country_region_code",
                     hover_name="country_region_code", size="retail_and_recreation_percent_change_from_baseline",
                     projection="natural earth")
fig.show()

ValueError: 
    Invalid element(s) received for the 'size' property of scattergeo.marker
        Invalid elements include: [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]

    The 'size' property is a number and may be specified as:
      - An int or float in the interval [0, inf]
      - A tuple, list, or one-dimensional numpy array of the above

In [3]:
px.data.gapminder()

,country,continent,year,lifeExp,pop,gdpPercap,iso_alpha,iso_num
0,Afghanistan,Asia,1952,28.801,8425333,779.445314,AFG,4
1,Afghanistan,Asia,1957,30.332,9240934,820.853030,AFG,4
2,Afghanistan,Asia,1962,31.997,10267083,853.100710,AFG,4
3,Afghanistan,Asia,1967,34.020,11537966,836.197138,AFG,4
4,Afghanistan,Asia,1972,36.088,13079460,739.981106,AFG,4
...,...,...,...,...,...,...,...,...
1699,Zimbabwe,Africa,1987,62.351,9216418,706.157306,ZWE,716
1700,Zimbabwe,Africa,1992,60.377,10704340,693.420786,ZWE,716
1701,Zimbabwe,Africa,1997,46.809,11404948,792.449960,ZWE,716
1702,Zimbabwe,Africa,2002,39.989,11926563,672.038623,ZWE,716
